# Beer Style Analysis

I want to take a look at some beer data and find meaningful insights about different types of beer. For this project, I plan to use two main data sources: web-scraping BeerAdvocate’s Top Rated tables and accessing a Kagglehub API of beer review data. Each source has its own drawbacks. BeerAdvocate only has one relevant table, which contains about 250 entries, while the Kagglehub API has a lot more data and information that can get overwhelming. I’ll start with some basic analysis of each data source on its own and then see if I can combine them in a meaningful way toward the end.

Let's start with data from Beer Advocate. We'll use BeautifulSoup to help us extract only the relevant data from the table. The site doesn't really have everything in it's own neat column, or really any headers at all, so we have some cleaning up to do. The biggest pain point is that the beer name, brewery, and beer style are all in the same `td` cell of the table, so we need to find which tags they are in, and extract them accordingly.

In [62]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://www.beeradvocate.com/beer/top-rated/"
headers = {"User-Agent": "Mozilla/5.0"}

soup = BeautifulSoup(requests.get(url, headers=headers).text, "html.parser")

rows = []

for tr in soup.find_all("tr"):
    tds = tr.find_all("td")

    if len(tds) != 5 or not tds[0].find("span"):
        continue

    # Grab the cell with beer name, brewery, style, and ABV
    beer_td = tds[1]

    # beer name
    beer_name = beer_td.find("b").get_text(strip=True)

    muted = beer_td.find("span", class_="muted")

    # brewery and style are links inside the muted span
    links = muted.find_all("a")
    brewery = links[0].get_text(strip=True)
    style = links[1].get_text(strip=True)

    # ABV is plain text after '|'
    abv_text = muted.get_text(strip=True).split("|")[-1]
    abv = float(abv_text.replace("%", ""))

    # Count of ratings is in it's own td
    num_ratings = int(tds[2].find("b").get_text(strip=True).replace(",", ""))


    # Average rating is in it's own td too
    avg_rating = float(tds[3].find("b").get_text(strip=True))

    rows.append({
        "beer_name": beer_name,
        "brewery": brewery,
        "style": style,
        "abv": abv,
        "num_ratings": num_ratings,
        "avg_rating": avg_rating
    })

ba_df = pd.DataFrame(rows)
ba_df.head(10)


,beer_name,brewery,style,abv,num_ratings,avg_rating
0,Kentucky Brunch Brand Stout,Toppling Goliath Brewing Company,Imperial Pastry Stout,12.0,955,4.83
1,O.W.K.,Side Project Brewing,American Imperial Stout,15.0,89,4.86
2,M.J.K.,Side Project Brewing,English Barleywine,20.0,93,4.85
3,Marshmallow Handjee,3 Floyds Brewing Co.,Imperial Pastry Stout,15.0,1834,4.73
4,Abraxas - Barrel-Aged,Perennial Artisan Ales,Imperial Pastry Stout,12.8,1727,4.73
5,R&D Sour Fruit (Very Sour Blackberry),New Glarus Brewing Company,Wild Ale,5.0,416,4.74
6,Hunahpu's Imperial Stout - Double Barrel Aged,Cigar City Brewing,American Imperial Stout,11.0,1673,4.71
7,Blessed,Anchorage Brewing Company,American Imperial Stout,15.5,312,4.73
8,Black Magick - Pappy Van Winkle,Voodoo Brewing Co.,American Imperial Stout,13.0,286,4.73
9,Heady Topper,The Alchemist,Imperial IPA,8.0,15400,4.70


Before we aggregate anything, let's see a nice visualization of our basic data. 

In [63]:
import plotly.express as px

fig = px.scatter(
    ba_df,
    x="num_ratings",
    y="avg_rating",
    size="abv",
    hover_data=["beer_name", "brewery", "style", "abv"],
    title="Top Rated Beers on BeerAdvocate",
    color="style",
    color_continuous_scale="Viridis",
    labels={
        "num_ratings": "Number of Ratings",
        "avg_rating": "Average Rating",
        "abv": "ABV (%)"
    }
)
fig.show()

We now have a nice dataframe for our Beer Advocate data. Let's do a little preliminary analysis to find out a little bit about beer types and ABV in the top 250. We'll group the table by the style of beer, using the mean of the individual average ratings, and taking the count of each one to see how many of each style make up the top 250. 

In [64]:
top250_style = (
    ba_df
    .groupby("style")
    .agg(
        avg_rating=("avg_rating", "mean"),
        n_top250=("beer_name", "count"),
        avg_abv =("abv", "mean"),
    )
    .reset_index()
)
count_t250 = top250_style.sort_values(by="n_top250", ascending=False)
avg_t250 = top250_style.sort_values(by="avg_rating", ascending=False)
abv_t250 = top250_style.sort_values(by="avg_abv", ascending=False)


pd.concat([count_t250.reset_index(drop=True), avg_t250.reset_index(drop=True), abv_t250.reset_index(drop=True)], axis=1)


,style,avg_rating,n_top250,avg_abv,style,avg_rating,n_top250,avg_abv,style,avg_rating,n_top250,avg_abv
0,American Imperial Stout,4.565532,47,13.572128,Belgian Lambic,4.700000,1,7.000000,American Barleywine,4.696667,3,17.286667
1,Hazy Imperial IPA,4.562750,40,8.477500,American Barleywine,4.696667,3,17.286667,American Strong Ale,4.520000,4,16.500000
2,Imperial Pastry Stout,4.599667,30,12.326667,English Barleywine,4.670000,7,14.438571,Rye Beer,4.630000,1,15.700000
3,Belgian Saison,4.580000,17,7.229412,Rye Beer,4.630000,1,15.700000,English Barleywine,4.670000,7,14.438571
4,Imperial IPA,4.566875,16,8.784375,Imperial Pastry Stout,4.599667,30,12.326667,Old Ale,4.550000,1,13.600000
5,Wild Ale,4.551333,15,6.836667,Belgian Saison,4.580000,17,7.229412,American Imperial Stout,4.565532,47,13.572128
6,Russian Imperial Stout,4.529091,11,12.645455,American Pale Ale,4.570000,1,6.500000,Russian Imperial Stout,4.529091,11,12.645455
7,Hazy IPA,4.550000,9,6.977778,Imperial IPA,4.566875,16,8.784375,Imperial Pastry Stout,4.599667,30,12.326667
8,Imperial Porter,4.556250,8,11.112500,American Imperial Stout,4.565532,47,13.572128,Imperial Porter,4.556250,8,11.112500
9,Belgian Fruit Lambic,4.550000,8,5.812500,Hazy Imperial IPA,4.562750,40,8.477500,Belgian Quadrupel (Quad),4.512500,4,10.950000


Using `pnadas` `concat()` we can see side by side comparison of the beer styles sorted by the amount that appear in the top 250 and sorted by average rating across the style. Seems like it could be interesting, but its tough to tell with just numbers. We need to visualize it. 

In [65]:


fig = px.scatter(
    top250_style,
    x="n_top250",
    y="avg_rating",
    size="avg_abv",
    hover_name="style",
    labels={
        "n_top250": "Top 250 Count",
        "avg_rating": "Average Rating",
        "avg_abv": "Average ABV (%)"
    },
    title="Beer Styles in Top 250: Count vs Average Rating"
)
fig.show()

We can see a couple interesting things from the data visualizations already. We can see that the lowest average ABV for a style is still almost 6%, with most being well above that. This tells us that the top 250 beers on Beer Advocate definitely skew towards the hgiiher end of the ABV spectrum. We can also see that asidie from what looks like a column of results towards the 0 end of the scale, average rating looks to have a slightly logarithmic shape to it as number of ratings increases. This shape holds for when ratings are averaged across styles. Let's work on fetching and doing a similar analysis on the Wine Vybe data via Kagglehub. I had to pivot from using Wine Vybe's API directly because they don't support fetching entire lists of data, you have to look up information specifically by beer (via it's id) or by brewery name. This doesn't really support our use case, so I went to a different source on Kaggle. Kaggle has it's own API to interact with, so I hope this qualifies.

Note: Make sure you get your own KaggleHub API key and add it to your own `.env` file. I know the project requirements state that it should be replicable code and results, but publishing API keys to Github is very bad practice.

In [66]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

file_path = "beer_profile_and_ratings.csv"


df = kagglehub.dataset_load(
  KaggleDatasetAdapter.PANDAS,
  "ruthgn/beer-profile-and-ratings-data-set",
  file_path
)
df = df.drop(columns=["review_aroma", "review_appearance", "review_palate", "review_taste", "Beer Name (Full)", "Description"])
# df["number_of_reviews", "Astringency", "Body", "Bitter", "Alcohol", "Sweet", "Sour", "Salty", "Fruits", "Hoppy", "Spices", "Malty"] = df["number_of_reviews", "Astringency", "Body", "Bitter", "Alcohol", "Sweet", "Sour", "Salty", "Fruits", "Hoppy", "Spices", "Malty"].apply(pd.to_numeric)
df = df.where(df['number_of_reviews'] >= 100).sort_values(by="review_overall", ascending=False)
df.head()


,Name,Style,Brewery,ABV,Min IBU,Max IBU,Astringency,Body,Alcohol,Bitter,Sweet,Sour,Salty,Fruits,Hoppy,Spices,Malty,review_overall,number_of_reviews
1606,Blåbær Lambik,Lambic - Fruit,Brasserie Cantillon,5.00,15.0,21.0,30.0,25.0,6.0,5.0,99.0,119.0,0.0,113.0,12.0,0.0,8.0,4.628205,156.0
959,Heady Topper,IPA - New England,The Alchemist,8.00,50.0,70.0,16.0,42.0,18.0,91.0,40.0,55.0,0.0,76.0,117.0,2.0,30.0,4.625800,469.0
2177,Trappist Westvleteren 12 (XII)Brouwerij Westvl...,Quadrupel (Quad),Brouwerij Westvleteren (Sint-Sixtusabdij van W...,10.20,25.0,50.0,5.0,44.0,49.0,14.0,86.0,21.0,0.0,53.0,5.0,12.0,57.0,4.617925,1272.0
944,Pliny The Younger,IPA - Imperial,Russian River Brewing Company,10.25,65.0,100.0,23.0,46.0,14.0,73.0,29.0,40.0,0.0,49.0,106.0,4.0,19.0,4.600000,610.0
2495,CBS (Canadian Breakfast Stout),Stout - American Imperial,Founders Brewing Company,11.30,50.0,80.0,3.0,116.0,57.0,47.0,112.0,4.0,1.0,8.0,18.0,21.0,150.0,4.591052,637.0


This look pretty interesting. I dropped some columns that I don't think we'll use as well as implemented a similar number of review min that Beer Advocate uses to ensure a beer is popular enough to count (I believe BA's is 150 but I used 100 to intentionally try to get a larger set). Let's see a similar first scatterplot.

In [67]:
fig = px.scatter(
    df,
    x="number_of_reviews",
    y="review_overall",
    size="ABV",
    hover_data=["Name", "Brewery", "Style", "ABV"],
    title="Top Rated Beers in Kaggle Dataset",
    color="Style",
    color_continuous_scale="Viridis",
)
fig.show()

This dataset clearly has a lot more data points. Still, we can see a fairly similar logarithmic shape to the data as the number of reviews grows. This suggests a pattern where the more popular a beer is, the more highly it is rated, with diminishing returns. Let's group by style again to see what this group of data reflects.


In [68]:
kaggle_style = (
    df
    .groupby("Style")
    .agg(
        avg_rating=("review_overall", "mean"),
        n_kaggle=("Name", "count"),
        avg_abv =("ABV", "mean"),
    )
    .reset_index()
)

kaggle_style.head()

,Style,avg_rating,n_kaggle,avg_abv
0,Altbier,3.888164,11,5.945455
1,Barleywine - American,3.867400,29,11.027586
2,Barleywine - English,3.902772,16,11.375000
3,Bitter - English,3.821963,10,4.820000
4,Bitter - English Extra Special / Strong Bitter...,3.889079,22,5.768182


In [69]:
fig = px.scatter(
    kaggle_style,
    x="n_kaggle",
    y="avg_rating",
    size="avg_abv",
    hover_name="Style",
    labels={
        "n_kaggle": "Count",
        "avg_rating": "Average Rating",
        "avg_abv": "Average ABV (%)"}
)

fig.show()

The shape of this data doesn't seem to match the logarithmic pattern we would expect. Other than a few outliers, it seems to be pretty groupd around the same average rating. We noted that the BA Top 250 list had a pretty high ABV value across the bored. Let's take this new set and see if there's a pattern with ABV and rating.

In [70]:
fig = px.scatter(
    df,
    x="ABV",
    y="review_overall",
    color="Style",
    hover_data=['Name', 'Brewery', 'ABV', 'review_overall'],
    title="ABV vs Overall Rating",
)
trend_fig = px.scatter(
    df,
    x="ABV",
    y="review_overall",
    trendline="ols"
)

fig.add_traces(trend_fig.data[1])

fig.show()

Above, we can see an interesting shape to the data. It's not quite logarithmic, but the lower ratings seem to happen more infrequently as ABV increases. We can see this reflected in a trendline that we extract from just an ABV vs Rating scatterplot (using a trendline on this visualization creates one for each beer type, which we don't want). The line trends gently upwards, and since it doesn't quite look like the data slopes upward, it must be do to the lack of lower reviews. This seems to suggest the pattern that a high ABV is not necessary for a high rating, but rather that higher ABV beers tend to be better rated on average.

It seems like we have two disticnt sets of data. The Beer Advocate data set describes specifically the top 250 best beers in their system as rated by their users. The Kaggle data set represents a larger swathe of beers and might be more representative of the beer landscape as a whole. Let's see if we can analyse what types of beer might be oversaturating the BA Top 250 market, and what beer types might be underrepresented. We can start by normalizing our data first. The very first step of this process is normalizing names of styles. We can see that each data set has different naming conventions, but we can broaden our range a bit. For instance, if a name includes "IPA" we can call it an IPA, and the same with Porter, Stout, etc. Let's write a reusable function to do this below.

In [ ]:
def normalize_style(style):
    style = style.lower()

    if "ipa" in style:
        return "IPA"
    elif "stout" in style:
        return "Stout"
    elif "porter" in style:
        return "Porter"
    elif "lager" in style:
        return "Lager"
    elif "pils" in style:
        return "Pilsner"
    elif "sour" in style or "gose" in style or "lambic" in style:
        return "Sour"
    elif "wheat" in style or "weiss" in style:
        return "Wheat"
    elif "belgian" in style:
        return "Belgian"
    elif "ale" in style:
        return "Ale"
    else:
        return "Other"


Now we can apply this to both data sets.

In [72]:
top250_style["style_group"] = top250_style["style"].apply(normalize_style)
kaggle_style["style_group"] = kaggle_style["Style"].apply(normalize_style)


Next, we can aggregate our data using the new groups.

In [73]:
ba_grouped = (
    top250_style
    .groupby("style_group")
    .agg(n_top250=("n_top250", "sum"))
    .reset_index()
)

kaggle_grouped = (
    kaggle_style
    .groupby("style_group")
    .agg(n_kaggle=("n_kaggle", "sum"))
    .reset_index()
)


After normalizing beer styles into broader categories, we aggregate each dataset by these categories and compute the proportion of beers that each style represents within its respective dataset. This allows us to compare how frequently a given style appears in BeerAdvocate’s Top 250 relative to how common it is in the larger Kaggle dataset. By dividing the Top 250 proportion by the overall proportion, we create a representation ratio, where values greater than 1 indicate styles that are overrepresented in the Top 250, and values less than 1 indicate styles that are underrepresented.

In [74]:
ba_grouped["ba_prop"] = ba_grouped.n_top250 / ba_grouped.n_top250.sum()
kaggle_grouped["kaggle_prop"] = kaggle_grouped.n_kaggle / kaggle_grouped.n_kaggle.sum()

style_compare = ba_grouped.merge(kaggle_grouped, on="style_group", how="inner")
style_compare["representation_ratio"] = (
    style_compare["ba_prop"] / style_compare["kaggle_prop"]
)


In [75]:

fig = px.bar(
    style_compare.sort_values("representation_ratio", ascending=False),
    x="representation_ratio",
    y="style_group",
    orientation="h",
    title="Style Representation in BeerAdvocate Top 250 vs Overall Dataset"
)

fig.show()


We can see that Porters, Ales, and Wheat beers are very under represented in the BA Top 250 while IPA and Stouts fill out a lot of the same list. Now that we know there is at least some level of discordance between the data sets, we can see if the styles that seem to fill the top 250 the same styles that are regularly rated highly in the larger data set. 

We can start by using the standard deviation to get a measure of consistency and then normalize it.

In [76]:
kaggle_consistency = (
    df
    .groupby("Style")
    .agg(
        avg_rating=("review_overall", "mean"),
        rating_std=("review_overall", "std"),
        n_beers=("Name", "count")
    )
    .reset_index()
)
kaggle_consistency["style_group"] = kaggle_consistency["Style"].apply(normalize_style)


Now we can aggregate again by group and merge the summaries.

In [77]:
kaggle_consistency_grouped = (
    kaggle_consistency
    .groupby("style_group")
    .agg(
        avg_rating=("avg_rating", "mean"),
        avg_std=("rating_std", "mean")
    )
    .reset_index()
)

final_compare = kaggle_consistency_grouped.merge(
    ba_grouped,
    on="style_group",
    how="inner"
)



fig = px.scatter(
    final_compare,
    x="avg_std",
    y="n_top250",
    hover_name="style_group",
    labels={
        "avg_std": "Rating Variability (Lower = More Consistent)",
        "n_top250": "Number of Top 250 Beers"
    },
    title="Beer Style Consistency vs Presence in Top 250"
)

fig.show()


This shows that both Stouts and IPA have more consistent ratings across the board, as well showing up in the top 250 more often. This suggests that these styles appear in the BA list because they are actually more highly rated on average. 

## Conclusion
Looking at these two datasets side by side helps highlight the difference between what rises to the very top of beer rating systems and what the broader beer landscape looks like overall. The BeerAdvocate Top 250 clearly skews toward higher-ABV beers and is dominated by a relatively small number of styles, particularly IPAs and Stouts. When compared against the much larger Kaggle dataset, these same styles not only appear disproportionately often in the Top 250, but also show more consistent ratings across a wider range of beers. This suggests that their prevalance in the top 250 is not purely the result of a few exceptional outliers, but rather reflects a broader pattern of consistently high ratings. At the same time, styles like Porters, Wheat beers, and Ales appear underrepresented in the Top 250 despite being common in the larger dataset, indicating that popularity and abundance do not necessarily translate to higher ratings. Overall, combining these two data sources provides a clearer picture of how beer styles perform both at the very top and across the wider beer ecosystem.